# Jupyter Notebook for Calculating Statistics

**NOTE**: the `json` files that contain overall metrics and also the linear regression model results were created manually.
This can be done with some Python code very easily. However, the code for this is not included in this notebook.

In [81]:
import pandas as pd
import numpy as np
from math import sqrt
import altair as alt
from utils import save_dataset_to_csv
import statsmodels.formula.api as smf
from tqdm import tqdm
import tiktoken

In [82]:
model_metadata = {
    "alpacare-7B": {"model_type": "biomedical open", "model_size_in_b": 7},
    "biomedgpt7B": {"model_type": "biomedical open", "model_size_in_b": 7},
    "biomistral7B": {"model_type": "biomedical open", "model_size_in_b": 7},
    "claude_3.5-haiku": {"model_type": "generalist closed", "model_size_in_b": None},
    "claude_3.5-sonnet": {"model_type": "generalist closed", "model_size_in_b": None},
    "gemini_1.5_flash": {"model_type": "generalist closed", "model_size_in_b": None},
    "gemini_1.5_flash-8B": {"model_type": "generalist closed", "model_size_in_b": 8},
    "gpt4o": {"model_type": "generalist closed", "model_size_in_b": None},
    "gpt4o-mini": {"model_type": "generalist closed", "model_size_in_b": None},
    "gpt35": {"model_type": "generalist closed", "model_size_in_b": 175},
    "llama2_chat-7B": {"model_type": "generalist open", "model_size_in_b": 7},
    "llama2_chat-13B": {"model_type": "generalist open", "model_size_in_b": 13},
    "llama2_chat-70B": {"model_type": "generalist open", "model_size_in_b": 70},
    "llama3_instruct-8B": {"model_type": "generalist open", "model_size_in_b": 8},
    "llama3_instruct-70B": {"model_type": "generalist open", "model_size_in_b": 70},
    "med42-8B": {"model_type": "biomedical open", "model_size_in_b": 8},
    "med42-70B": {"model_type": "biomedical open", "model_size_in_b": 70},
    "mistral_instruct7B": {"model_type": "generalist open", "model_size_in_b": 7},
    "olmo2_instruct-7B": {"model_type": "generalist open", "model_size_in_b": 7},
    "olmo2_instruct-13B": {"model_type": "generalist open", "model_size_in_b": 13},
    "openbiollm-8B": {"model_type": "biomedical open", "model_size_in_b": 8},
    "openbiollm-70B": {"model_type": "biomedical open", "model_size_in_b": 70}
}

## Spin Detection Task

In [ ]:
# detection_overall_metrics.json was manually created
detection_stats_df = pd.read_json("./eval_outputs/detection_overall_metrics.json", orient="index")

detection_stats_df["model_name"] = detection_stats_df.index
detection_stats_df["model_type"] = detection_stats_df.index.map(lambda x: model_metadata[x]["model_type"])
detection_stats_df["model_size_in_b"] = detection_stats_df.index.map(lambda x: model_metadata[x]["model_size_in_b"])
# remove index
detection_stats_df.reset_index(drop=True, inplace=True)

print(f"Number of models: {len(detection_stats_df)}")

detection_stats_df.sort_index(inplace=True) # alphabetical order
detection_stats_df

### Average of accuracy by model type

In [ ]:
# Group by model type and calculate mean accuracy and standard deviation
accuracy_by_model_type = detection_stats_df.groupby('model_type')['accuracy'].agg(['mean', 'std']).reset_index()

# Rename columns for clarity
accuracy_by_model_type.columns = ['model_type', 'mean_accuracy', 'std_deviation']

print(accuracy_by_model_type)

#### Plots

In [ ]:
# Create a dictionary for custom labels
custom_labels = {
    "alpacare-7B": "AlpaCare 7B",
    "biomedgpt7B": "BioMedGPT 7B",
    "biomistral7B": "BioMistral 7B",
    "claude_3.5-haiku": "Claude3.5 Haiku",
    "claude_3.5-sonnet": "Claude3.5 Sonnet", 
    "gemini_1.5_flash": "Gemini1.5 Flash",
    "gemini_1.5_flash-8B": "Gemini1.5 Flash 8B",
    "gpt4o": "GPT4o",
    "gpt4o-mini": "GPT4o Mini",
    "gpt35": "GPT3.5",
    "llama2_chat-7B": "Llama2 Chat 7B",
    "llama2_chat-13B": "Llama2 Chat 13B",
    "llama2_chat-70B": "Llama2 Chat 70B",
    "llama3_instruct-8B": "Llama3 Instruct 8B",
    "llama3_instruct-70B": "Llama3 Instruct 70B",
    "med42-8B": "Med42 8B",
    "med42-70B": "Med42 70B",
    "mistral_instruct7B": "Mistral Instruct 7B",
    "olmo2_instruct-7B": "Olmo2 Instruct 7B",
    "olmo2_instruct-13B": "Olmo2 Instruct 13B",
    "openbiollm-8B": "OpenBioLLM 8B",
    "openbiollm-70B": "OpenBioLLM 70B"
}

detection_stats_df['model_name_custom'] = detection_stats_df['model_name'].map(custom_labels)

color_mapping = {
    'biomedical open': '#0868ac', 
    'generalist closed': '#7bccc4',
    'generalist open': '#bae4bc',
}

# Create the bar chart
chart = alt.Chart(detection_stats_df).mark_bar().encode(
    y=alt.Y('model_name_custom:N', sort='-x', title='Model Name'),
    x=alt.X('accuracy:Q', title='Accuracy'),
    color=alt.Color('model_type:N', title='Model Type',
                    scale=alt.Scale(domain=list(color_mapping.keys()), range=list(color_mapping.values())),
                    legend=alt.Legend(
                    orient='none',
                    legendX=130, legendY=-45,
                    direction='horizontal',
                    titleAnchor='middle'))  # Legend at the bottom
).properties(
    width=800,
)

# Add value labels with increased font size
text = chart.mark_text(
    align='center',
    baseline='middle',
    fontWeight='bold',
    dx=20  # Adjust the position of the text
).encode(
    text=alt.Text('accuracy:Q', format='.2f'),
    color=alt.value('black'),
)

# Add a mean rule
avg_rule = alt.Chart(detection_stats_df).mark_rule(color='red').encode(
    x='mean(accuracy):Q',
    size=alt.value(2)
)

# Add a 50% chance rule
chance_rule = alt.Chart(detection_stats_df).mark_rule(color='gray').encode(
    x='min(accuracy):Q',
    size=alt.value(2),
    strokeDash=alt.value([10, 10])
)

# Increase font size for axis labels, titles, and other components
chart_config = {
    "axis": {"labelFontSize": 20, "titleFontSize": 22},  # Axis labels and titles
    "header": {"labelFontSize": 20, "titleFontSize": 22},  # Title and facet headers (if any)
    "legend": {"labelFontSize": 18, "titleFontSize": 20},  # Legend labels and titles
    "text": {"fontSize": 20},  # Text mark size
}

# Combine chart and text, and apply the config
c_t = chart + avg_rule + chance_rule + text
c_t = c_t.configure(**chart_config)  # Apply the global configuration

# Save to HTML
c_t.save("./plots/detection_accuracy_by_model.html")

# Display the chart
c_t

In [ ]:
# Plot average accuracy by model_type and add error bars
bars = alt.Chart(detection_stats_df).mark_bar().encode(
    x=alt.X('model_type:N', title='Model Type', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('mean(accuracy):Q', title='Mean Accuracy'),
    color=alt.Color('model_type:N', title='Model Type', legend=None)
).properties(
    title='Average Accuracy by Model Type',
    width=800  # Set the width to 800 pixels
)

error_bars = alt.Chart(detection_stats_df).mark_errorbar(extent='stdev').encode(
    x=alt.X('model_type:N'),
    y=alt.Y('accuracy:Q')
)

# Add value labels
text = bars.mark_text(
    align='center',
    baseline='bottom',
    dy=-5  # Adjust the position of the text
).encode(
    text=alt.Text('mean(accuracy):Q', format='.2f')
)

alt.layer(bars, error_bars, text)

### Average of accuracy by model size

In [91]:
# model size in buckets (0-10B, 11-20B, 22-100B, 100B+/NaN)
def model_size_bucket(model_size): 
    if model_size is None or pd.isna(model_size):
        return "Unknown"
    elif model_size >= 100:
        return "100B+"
    elif model_size <= 10:
        return "0-10B"
    elif model_size <= 20:
        return "11-20B"
    else:
        return "21-100B"

In [ ]:
# average accuracy by model size
detection_stats_df["model_size_bucket"] = detection_stats_df["model_size_in_b"].map(model_size_bucket)

accuracy_by_model_size = detection_stats_df.groupby('model_size_bucket')['accuracy'].agg(['mean', 'std']).reset_index()

# Rename columns for clarity
accuracy_by_model_size.columns = ['model_size_bucket', 'mean_accuracy', 'std_deviation']

print(accuracy_by_model_size)

#### Plots

In [ ]:
bars = alt.Chart(detection_stats_df).mark_bar().encode(
    x=alt.X('model_name:N', sort='-y', title='Model Name'),
    y=alt.Y('accuracy:Q', title='Accuracy'),
    color=alt.Color('model_size_bucket:N', title='Model Size Bucket')
).properties(
    title='Accuracy by Model Size Bucket',
    width=800,
)

# Add value labels
text = bars.mark_text(
    align='center',
    baseline='bottom',
    dy=-5  # Adjust the position of the text
).encode(
    text=alt.Text('accuracy:Q', format='.2f')
)

bars + text

## RCT Trial Result Interpretation Task

In [ ]:
# interpretation_overall_metrics.json was manually created
interpretation_stats_df = pd.read_json("./eval_outputs/interpretation_overall_metrics.json", orient="index")

interpretation_stats_df["model_name"] = interpretation_stats_df.index
interpretation_stats_df["model_type"] = interpretation_stats_df.index.map(lambda x: model_metadata[x]["model_type"])
interpretation_stats_df["model_size_in_b"] = interpretation_stats_df.index.map(lambda x: model_metadata[x]["model_size_in_b"])
# remove index
interpretation_stats_df.reset_index(drop=True, inplace=True)

print(f"Number of models: {len(interpretation_stats_df)}")

interpretation_stats_df.sort_index(inplace=True) # alphabetical order
interpretation_stats_df

In [ ]:
human_expert_stats = {
        "benefit_answer": {"mean_diff": 0.71, "ci_lower": 0.07, "ci_upper": 1.35},
        "rigor_answer": {"mean_diff": -0.59, "ci_lower": -1.13, "ci_upper": -0.05},
        "importance_answer": {"mean_diff": -0.38, "ci_lower": -0.95, "ci_upper": 0.19},
        "full_text_answer": {"mean_diff": 0.77, "ci_lower": 0.08, "ci_upper": 1.47},
        "another_trial_answer": {"mean_diff": 0.64, "ci_lower": -0.03, "ci_upper": 1.31}
    }

human_expert_stats_df = pd.DataFrame(human_expert_stats).T
human_expert_stats_df["metric"] = human_expert_stats_df.index
# remove index
human_expert_stats_df.reset_index(drop=True, inplace=True)
human_expert_stats_df["method"] = "human experts"

human_expert_stats_df

In [101]:
def calculate_confidence_interval(df, df_column_name):
    mean_diff = df[df_column_name].mean()  # Calculate the mean
    std_dev = df[df_column_name].std()  # Calculate the standard deviation
    n = len(df[df_column_name])  # Sample size

    # Calculate the margin of error for 95% CI (z = 1.96)
    z = 1.96
    margin_of_error = z * (std_dev / sqrt(n))

    # Calculate the 95% Confidence Interval
    ci_lower = mean_diff - margin_of_error
    ci_upper = mean_diff + margin_of_error

    return ci_lower, ci_upper

In [102]:
def calculate_model_stats(interpretation_stats_df, method_name = "all LLMs"):
    # calculate the average of all model metrics and calculate 95% CI
    average_model_benefit = interpretation_stats_df["benefit_answer_mean_diff"].mean()
    ci_lower_model_benefit, ci_upper_model_benefit = calculate_confidence_interval(interpretation_stats_df, "benefit_answer_mean_diff")

    average_model_rigor = interpretation_stats_df["rigor_answer_mean_diff"].mean()
    ci_lower_model_rigor, ci_upper_model_rigor = calculate_confidence_interval(interpretation_stats_df, "rigor_answer_mean_diff")

    average_model_importance = interpretation_stats_df["importance_answer_mean_diff"].mean()
    ci_lower_model_importance, ci_upper_model_importance = calculate_confidence_interval(interpretation_stats_df, "importance_answer_mean_diff")

    average_model_full_text = interpretation_stats_df["full_text_answer_mean_diff"].mean()
    ci_lower_model_full_text, ci_upper_model_full_text = calculate_confidence_interval(interpretation_stats_df, "full_text_answer_mean_diff")

    average_model_another_trial = interpretation_stats_df["another_trial_answer_mean_diff"].mean()
    ci_lower_model_another_trial, ci_upper_model_another_trial = calculate_confidence_interval(interpretation_stats_df, "another_trial_answer_mean_diff")

    model_stats = {
        "benefit_answer": {"mean_diff": average_model_benefit, "ci_lower": ci_lower_model_benefit, "ci_upper": ci_upper_model_benefit},
        "rigor_answer": {"mean_diff": average_model_rigor, "ci_lower": ci_lower_model_rigor, "ci_upper": ci_upper_model_rigor},
        "importance_answer": {"mean_diff": average_model_importance, "ci_lower": ci_lower_model_importance, "ci_upper": ci_upper_model_importance},
        "full_text_answer": {"mean_diff": average_model_full_text, "ci_lower": ci_lower_model_full_text, "ci_upper": ci_upper_model_full_text},
        "another_trial_answer": {"mean_diff": average_model_another_trial, "ci_lower": ci_lower_model_another_trial, "ci_upper": ci_upper_model_another_trial}
    }

    model_stats_df = pd.DataFrame(model_stats).T
    model_stats_df["metric"] = model_stats_df.index
    # remove index
    model_stats_df.reset_index(drop=True, inplace=True)
    model_stats_df["method"] = method_name

    return model_stats_df

In [ ]:
# calculate the average of all model metrics and calculate 95% CI
model_stats_df = calculate_model_stats(interpretation_stats_df)

model_stats_df

In [104]:
# get average and 95% CI by model_type from interpretation_stats_df for each model_type
interpretation_stats_df["model_type"] = interpretation_stats_df["model_name"].map(lambda x: model_metadata[x]["model_type"])

# get only generalist closed  models
generalist_closed_model_stats = interpretation_stats_df[interpretation_stats_df["model_type"] == "generalist closed"]

# get only generalist open models
generalist_open_model_stats = interpretation_stats_df[interpretation_stats_df["model_type"] == "generalist open"]

# get only biomedical open models
biomedical_open_model_stats = interpretation_stats_df[interpretation_stats_df["model_type"] == "biomedical open"]


In [105]:
generalist_closed_model_stats_df = calculate_model_stats(generalist_closed_model_stats, method_name = "generalist closed")

generalist_open_model_stats_df = calculate_model_stats(generalist_open_model_stats, method_name = "generalist open")

biomedical_open_model_stats_df = calculate_model_stats(biomedical_open_model_stats, method_name = "biomedical open")

In [ ]:
generalist_closed_model_stats_df

In [ ]:
generalist_open_model_stats_df

In [ ]:
biomedical_open_model_stats_df

In [ ]:
average_by_model_type = pd.concat([generalist_closed_model_stats_df, generalist_open_model_stats_df, biomedical_open_model_stats_df], ignore_index=True)

average_by_model_type

In [ ]:
#combine all the dataframes
model_stats_final_df = pd.concat([human_expert_stats_df, model_stats_df, average_by_model_type], ignore_index=True)
#drop "_answer" from the values in metric column
model_stats_final_df['metric'] = model_stats_final_df['metric'].str.replace('_answer', '')

model_stats_final_df

### Plots

In [ ]:
# Create a mapping for custom facet titles
facet_title_mapping = {
    'benefit': 'Treatment Benefit',
    'rigor': 'Study Rigor',
    'importance': 'Study Importance',
    'full_text': 'Interest to Read Full-Text',
    'another_trial': 'Interest to Run Another Trial'
}

# Define the desired order for the facets
facet_order = ['Treatment Benefit', 'Study Rigor', 'Study Importance', 'Interest to Read Full-Text', 'Interest to Run Another Trial']

color_mapping = {
    'human experts': '#0868ac', 
    'all LLMs': '#43a2ca',  
    'generalist closed': '#7bccc4',  
    'generalist open': '#bae4bc',  
    'biomedical open': '#E3F4D4'
}

method_order = ['human experts', 'all LLMs', 'generalist closed', 'generalist open', 'biomedical open']

# Apply the mapping as a calculated field
chart_data = model_stats_final_df.copy()
chart_data['metric'] = chart_data['metric'].map(facet_title_mapping)

# Configure global font sizes
chart_config = {
    "axis": {"labelFontSize": 20, "titleFontSize": 22},  # Axis labels and titles
    "header": {"labelFontSize": 20, "titleFontSize": 22},  # Facet headers
    "legend": {"labelFontSize": 18, "titleFontSize": 20},  # Legend labels and titles
    "text": {"fontSize": 20},  # Text mark size
}

# Bar chart
bars = alt.Chart(chart_data).mark_bar().encode(
    x=alt.X('method:N', title=None, axis=alt.Axis(labelAngle=-45), sort=method_order),
    y=alt.Y('mean_diff:Q', title='Mean Difference'),
    color=alt.Color('method:N', title='Method', legend=None, scale=alt.Scale(domain=list(color_mapping.keys()), range=list(color_mapping.values())))
).properties(
    width=300,  # Set the width to 300 pixels
    height=300  # Set the height to 300 pixels
)

# Error bars
error_bars = alt.Chart(chart_data).mark_errorbar().encode(
    alt.X("method:N", sort=method_order),
    alt.Y("ci_lower:Q").title("Mean Difference"),
    alt.Y2("ci_upper:Q"),
    strokeWidth=alt.value(2),
    color=alt.value('gray')
)

# Add value labels
text = bars.mark_text(
    align='center',
    baseline='bottom',
    fontWeight='bold',
    dy=alt.expr(expr=alt.expr.if_(alt.datum.mean_diff >= 0, -1, 20))  # Adjust the position of the text    
).encode(
    text=alt.Text('mean_diff:Q', format='.2f'),
    color=alt.value('black')  # Set text color to black
)

# Combine layers and facet
chart = alt.layer(bars, error_bars, text, data=chart_data).facet(
    column=alt.Column('metric:N', title=None, sort=facet_order),
).configure(**chart_config)  # Apply the global configuration

# save to html
chart.save("./plots/interpretation_by_measures.html")

chart

### Mitigation Strategies Results

In [112]:
model_stats_df['method'] = 'baseline'

In [ ]:
# gold_labelled_interpretation_overall_metrics.json was manually created
gold_labelled_interpretation_stats_df = pd.read_json("./eval_outputs/gold_labelled_interpretation_overall_metrics.json", orient="index")

gold_labelled_interpretation_stats_df["model_name"] = gold_labelled_interpretation_stats_df.index
gold_labelled_interpretation_stats_df["model_type"] = gold_labelled_interpretation_stats_df.index.map(lambda x: model_metadata[x]["model_type"])
gold_labelled_interpretation_stats_df["model_size_in_b"] = gold_labelled_interpretation_stats_df.index.map(lambda x: model_metadata[x]["model_size_in_b"])
# remove index
gold_labelled_interpretation_stats_df.reset_index(drop=True, inplace=True)

print(f"Number of models: {len(gold_labelled_interpretation_stats_df)}")

gold_labelled_interpretation_stats_df.sort_index(inplace=True) # alphabetical order

In [ ]:
# calculate the average of all model metrics and calculate 95% CI
gold_labelled_model_stats_df = calculate_model_stats(gold_labelled_interpretation_stats_df, method_name = "+ ref labels")
gold_labelled_model_stats_df

In [ ]:
# model_output_labelled_interpretation_overall_metrics.json was manually created
model_output_labelled_interpretation_stats_df = pd.read_json("./eval_outputs/model_output_labelled_interpretation_overall_metrics.json", orient="index")

model_output_labelled_interpretation_stats_df["model_name"] = model_output_labelled_interpretation_stats_df.index
model_output_labelled_interpretation_stats_df["model_type"] = model_output_labelled_interpretation_stats_df.index.map(lambda x: model_metadata[x]["model_type"])
model_output_labelled_interpretation_stats_df["model_size_in_b"] = model_output_labelled_interpretation_stats_df.index.map(lambda x: model_metadata[x]["model_size_in_b"])
# remove index
model_output_labelled_interpretation_stats_df.reset_index(drop=True, inplace=True)

print(f"Number of models: {len(model_output_labelled_interpretation_stats_df)}")

model_output_labelled_interpretation_stats_df.sort_index(inplace=True) # alphabetical order

In [ ]:
# calculate the average of all model metrics and calculate 95% CI
model_output_labelled_model_stats_df = calculate_model_stats(model_output_labelled_interpretation_stats_df, method_name = "+ model labels")
model_output_labelled_model_stats_df

In [ ]:
# combined_detection_interpretation_overall_metrics.json was manually created
combined_interpretation_stats_df = pd.read_json("./eval_outputs/combined_detection_interpretation_overall_metrics.json", orient="index")

combined_interpretation_stats_df["model_name"] = combined_interpretation_stats_df.index
combined_interpretation_stats_df["model_type"] = combined_interpretation_stats_df.index.map(lambda x: model_metadata[x]["model_type"])
combined_interpretation_stats_df["model_size_in_b"] = combined_interpretation_stats_df.index.map(lambda x: model_metadata[x]["model_size_in_b"])
# remove index
combined_interpretation_stats_df.reset_index(drop=True, inplace=True)

print(f"Number of models: {len(combined_interpretation_stats_df)}")

combined_interpretation_stats_df.sort_index(inplace=True) # alphabetical order
# combined_interpretation_stats_df

In [ ]:
# calculate the average of all model metrics and calculate 95% CI
combined_interpretation_stats_df = calculate_model_stats(combined_interpretation_stats_df, method_name = "detect + interpret")
combined_interpretation_stats_df

In [ ]:
all_results = pd.concat([human_expert_stats_df, model_stats_df, gold_labelled_model_stats_df, model_output_labelled_model_stats_df, combined_interpretation_stats_df], ignore_index=True)

all_results

In [ ]:
# Create a mapping for custom facet titles
facet_title_mapping = {
    'benefit_answer': 'Treatment Benefit',
    'rigor_answer': 'Study Rigor',
    'importance_answer': 'Study Importance',
    'full_text_answer': 'Interest to Read Full-Text',
    'another_trial_answer': 'Interest to Run Another Trial'
}

# Define the desired order for the facets
facet_order = ['Treatment Benefit', 'Study Rigor', 'Study Importance', 'Interest to Read Full-Text', 'Interest to Run Another Trial']

color_mapping = {
    'human experts': '#0868ac',  
    'baseline': '#43a2ca',  
    '+ ref labels': '#7bccc4',  
    '+ model labels': '#bae4bc', 
    'detect + interpret': '#E3F4D4'  
}

method_order = ['human experts', 'baseline', '+ ref labels', '+ model labels', 'detect + interpret']

# Apply the mapping as a calculated field
chart_data = all_results.copy()
chart_data['metric'] = chart_data['metric'].map(facet_title_mapping)

# Configure global font sizes
chart_config = {
    "axis": {"labelFontSize": 20, "titleFontSize": 22},  # Axis labels and titles
    "header": {"labelFontSize": 20, "titleFontSize": 22},  # Facet headers
    "legend": {"labelFontSize": 18, "titleFontSize": 20},  # Legend labels and titles
    "text": {"fontSize": 20},  # Text mark size
}

# Bar chart
bars = alt.Chart(chart_data).mark_bar().encode(
    x=alt.X('method:N', title=None, axis=alt.Axis(labelAngle=-45), sort = method_order),
    y=alt.Y('mean_diff:Q', title='Mean Difference'),
    color=alt.Color('method:N', title='Method', legend=None, scale=alt.Scale(domain=list(color_mapping.keys()), range=list(color_mapping.values())))
).properties(
    width=300,  # Set the width to 300 pixels
    height=300  # Set the height to 300 pixels
)

# Error bars
error_bars = alt.Chart(chart_data).mark_errorbar().encode(
    alt.X("method:N", sort = method_order),
    alt.Y("ci_lower:Q").title("Mean Difference"),
    alt.Y2("ci_upper:Q"),
    strokeWidth=alt.value(2),
    color=alt.value('gray')
)

# Add value labels
text = bars.mark_text(
    align='center',
    baseline='bottom',
    fontWeight='bold',
    dy=alt.expr(expr=alt.expr.if_(alt.datum.mean_diff >= 0, -1, 20))  # Adjust the position of the text    
).encode(
    text=alt.Text('mean_diff:Q', format='.2f'),
    color=alt.value('black')  # Set text color to black
)

# Combine layers and facet
chart = alt.layer(bars, error_bars, text, data=chart_data).facet(
    column=alt.Column('metric:N', title=None, sort=facet_order),
).configure(**chart_config)  # Apply the global configuration

# save to html
chart.save("./plots/interpretation_by_measures_all_methods.html")

chart

## Relationship between spin detection and spin interpretation

Linear Regression with statsmodels Python package

In [ ]:
# get all model names
model_names = model_metadata.keys()
# remove alpacare-13B
model_names = [x for x in model_names if x != "alpacare-13B"]

len(model_names)

In [124]:
measures = ["benefit_answer", "rigor_answer", "importance_answer", "full_text_answer", "another_trial_answer"]
gpt_models = ["gpt4o", "gpt4o-mini", "gpt35"]
huggingface_models = ["alpacare-7B", "biomedgpt7B", "biomistral7B", 
                      "llama2_chat-7B", "llama2_chat-13B", "llama2_chat-70B",
                      "llama3_instruct-8B", "llama3_instruct-70B",
                      "med42-8B", "med42-70B", "mistral_instruct7B", 
                      "olmo2_instruct-7B", "olmo2_instruct-13B",
                      "openbiollm-8B", "openbiollm-70B"]
no_probability_models = ["claude_3.5-haiku", "claude_3.5-sonnet", "gemini_1.5_flash", "gemini_1.5_flash-8B"]

def get_is_detection_correct(row):
    if row['abstract_type'] == "spin":
        return row['model_answer'] == "yes"
    else:
        return row['model_answer'] == "no"
    
def get_is_abstract_type_spin(row):
    return row['abstract_type'] == "spin"
    
def detection_probability_gpt(row):
    # find the first instance of "yes" or "no"
    token_probabilties = row['model_log_probabilities']
    for token_prob in token_probabilties:
        if token_prob['token'].lower() == "yes":
            return np.exp(token_prob['logprob'])
        elif token_prob['token'].lower() == "no":
            return np.exp(token_prob['logprob'])
    return None # this should not happen but just in case

def detection_probability_huggingface(row):
    # find the first instance of "yes" or "no"
    token_probabilties = row['model_log_probabilities']
    for token_prob in token_probabilties:
        if token_prob['token_string'].lower() == "yes":
            return token_prob['probability']
        elif token_prob['token_string'].lower() == "no":
            return token_prob['probability']
    return None # this should not happen but just in case


def prepare_data_for_regression(model_names):
    for model_name in tqdm(model_names):
        # print(f"Processing {model_name}...")
        final_data = []
        detection_output_file_path = f"./eval_outputs/{model_name}/{model_name}_detection_outputs.json"
        interpretation_output_file_path = f"./eval_outputs/{model_name}/{model_name}_interpretation_outputs.json"
        model_detection_data = pd.read_json(detection_output_file_path, orient="records")
        model_interpretation_data = pd.read_json(interpretation_output_file_path, orient="records")

        # merge model_detection_data and model_interpretation_data by PMID and abstract_type
        model_data = pd.merge(model_detection_data, model_interpretation_data, on=['PMID', 'abstract_type'])

        # loop through each row in model_data
        for _, row in model_data.iterrows():
            detection_model_prediction = 1 if row['model_answer'] == "yes" else 0
            is_detection_correct = 1 if get_is_detection_correct(row) else 0
            is_spin_in_abstract = 1 if get_is_abstract_type_spin(row) else 0

            if model_name in gpt_models:
                detection_probability = detection_probability_gpt(row)
            elif model_name in huggingface_models:
                detection_probability = detection_probability_huggingface(row)
            else:
                detection_probability = None
            
            for measure in measures:
                final_data.append({
                    "pmid": row['PMID'],
                    "measure": measure,
                    "is_spin_in_abstract": is_spin_in_abstract,
                    "is_detection_correct": is_detection_correct,
                    "detection_model_prediction": detection_model_prediction,
                    "detection_probability": detection_probability,
                    "interpretation_answer": float(row[measure]) if row[measure] != "" else None
                })
            # calculate the average of the differences
            answers = []
            for measure in measures:
                if row[measure] != "":
                    answers.append(float(row[measure]))
            if len(answers) > 0:
                avg_answer= round(np.mean(answers), 6)
            else:
                avg_answer = None
            # add the average difference to the data
            final_data.append({
                "pmid": row['PMID'],
                "measure": "overall",
                "is_spin_in_abstract": is_spin_in_abstract,
                "is_detection_correct": is_detection_correct,
                "detection_model_prediction": detection_model_prediction,
                "detection_probability": detection_probability,
                "interpretation_answer": avg_answer
            })

        # save the final data to a json file
        json_file_path = f"./eval_outputs/{model_name}/{model_name}_combined_data.csv"
        save_dataset_to_csv(final_data, json_file_path)

In [ ]:
prepare_data_for_regression(model_names=model_names)

#### Simplest Regression

Is spin in abstract and the measures answers

In [126]:
for model_name in model_names:
    output_string = ""
    csv_file_path = f"./eval_outputs/{model_name}/{model_name}_combined_data.csv"
    data = pd.read_csv(csv_file_path)

    measures = ["benefit_answer", "rigor_answer", "importance_answer", "full_text_answer", "another_trial_answer", "overall"]
    for measure in measures:
        # get the data for the current measure
        measure_data = data[data['measure'] == measure]
        nan_rows_number = measure_data['interpretation_answer'].isnull().sum()
        # remove rows with NaN values in interpretation_answer
        measure_data = measure_data.dropna(subset=['interpretation_answer'])

        # check if there are less than 2 rows
        if len(measure_data) < 2:
            continue
        
        # fit the model
        model = smf.ols(formula="interpretation_answer ~ is_spin_in_abstract", 
                                    data=measure_data)
        results = model.fit()

        output_string += f"Model: {model_name} - {measure}\n"
        # print number of rows with NaN value(s)
        output_string += f"Number of rows with NaN value(s) in {model_name}: {nan_rows_number}\n"
        output_string += results.summary().as_text()
        output_string += "\n"

    # save the model summary
    with open(f"./eval_outputs/{model_name}/{model_name}_simple_regression_summary.txt", "w") as f:
        f.write(output_string)

##### Forest Plot for "Benefit" Linear Regression Results

In [ ]:
import altair as alt
import pandas as pd

# Load the JSON data into a DataFrame
# this json file was manually created
regression_results_df = pd.read_json("./eval_outputs/simple_linear_regression_benefit_data.json", orient="index")

# Ensure index is reset and available as a column
regression_results_df.reset_index(inplace=True)
regression_results_df = regression_results_df.rename(columns={'index': 'model_name'})

regression_results_df["model_name_custom"] = regression_results_df["model_name"].map(custom_labels)

# Create the Altair chart
points = alt.Chart(regression_results_df).mark_point(
    filled=True,
    color='red',
    size=50  # Increase point size
).encode(
    x=alt.X('coef:Q').title('Coefficient'),
    y=alt.Y('model_name_custom:N').title('LLM Name').sort(
        field='coef',  # Sort by coefficient values
        order='descending'
    )
).properties(
    width=600,
    height=300
)

# Add error bars
error_bars = points.mark_rule(
    strokeWidth=2  # Increase width of error bars
).encode(
    x='ci_lower:Q',
    x2='ci_upper:Q',
    size=alt.value(2)  # Set the width of error bars
)

# Add vertical line at x = 0.71
vertical_line = alt.Chart(pd.DataFrame({'x': [0.71]})).mark_rule(
    color='blue',
    strokeDash=[4, 4],  # Make it dashed
    strokeWidth=2
).encode(
    x='x:Q',
    color=alt.value('#0868ac')  # Specify the color directly
)

# Add label for vertical line
label = alt.Chart(pd.DataFrame({'x': [0.71], 'y': [regression_results_df['model_name_custom'].iloc[0]]})).mark_text(
    text='Human Experts',
    align='center',
    dx=5,  # Adjust text position
    dy=-10,
    fontSize=14,
    fontWeight='bold',
).encode(
    x='x:Q',
    y=alt.value(0),  # Adjust position if necessary
    color=alt.value('#0868ac')  # Specify the color directly
)

# Define custom x-axis labels
custom_labels_df = pd.DataFrame({
    'x': [regression_results_df['coef'].min(), regression_results_df['coef'].max()],
    'text': ['Less susceptible to spin', 'More susceptible to spin']
})

# Define custom x-axis labels
left_arrow_df = pd.DataFrame({
    'x': [0.2],
    'text': ['←']
})

# Define custom x-axis labels
right_arrow_df = pd.DataFrame({
    'x': [7.7],
    'text': ['→']
})

# Create a text layer for custom x-axis labels
custom_x_labels = alt.Chart(custom_labels_df).mark_text(
    align='center',
    baseline='bottom',
    dy=195,  # Adjust vertical positioning
    dx=24, # adjust horizontal positioning
    fontSize=14,
    fontWeight='bold',
).encode(
    x='x:Q',
    text='text:N',
    color=alt.value('#0868ac')  # Specify the color directly
)

# Create a text layer for custom x-axis labels
custom_x_left_arrow = alt.Chart(left_arrow_df).mark_text(
    align='center',
    baseline='bottom',
    dy=195,  # Adjust vertical positioning
    dx=10, # adjust horizontal positioning
    fontSize=24,
    fontWeight='bold',
).encode(
    x='x:Q',
    text='text:N',
    color=alt.value('#0868ac')  # Specify the color directly
)

# Create a text layer for custom x-axis labels
custom_x_right_arrow = alt.Chart(right_arrow_df).mark_text(
    align='center',
    baseline='bottom',
    dy=195,  # Adjust vertical positioning
    dx=0, # adjust horizontal positioning
    fontSize=24,
    fontWeight='bold',
).encode(
    x='x:Q',
    text='text:N',
    color=alt.value('#0868ac')  # Specify the color directly
)

# Combine all layers, including the new x-axis labels
chart = alt.layer(error_bars, points, vertical_line, label, custom_x_labels, custom_x_left_arrow, custom_x_right_arrow).configure_axis(
    labelFontSize=16,
    titleFontSize=18
).configure_title(
    fontSize=20
)

# # Save to HTML
chart.save("./plots/simple_regression_benefit_data.html")

chart


## PLS LLM Interpretations

### Average Tokens

In [ ]:
# get all PLS outputs from all LLMs
enc = tiktoken.get_encoding("o200k_base") # for gpt-4o and gpt-4o mini

model_token_stats = {}
number_of_tokens_total = [] # for all models
for model_name in model_names:
    csv_file_path = f"./pls_outputs/{model_name}/{model_name}_outputs.csv"
    data = pd.read_csv(csv_file_path)
    # calculate the number of tokens for each row in plain_language_summary
    plain_language_summaries = data['plain_language_summary'].tolist()

    number_of_tokens = []
    for summary in plain_language_summaries:
        token_integers = enc.encode(summary)
        number_of_tokens.append(len(token_integers))
        number_of_tokens_total.append(len(token_integers))

    # average number of tokens
    average_number_of_tokens = np.mean(number_of_tokens)
    # SD of tokens
    sd_number_of_tokens = np.std(number_of_tokens)
    model_token_stats[model_name] = {"average_number_of_tokens": average_number_of_tokens, "sd_number_of_tokens": sd_number_of_tokens}

model_token_stats_df = pd.DataFrame(model_token_stats).T
model_token_stats_df["model_name"] = model_token_stats_df.index

model_token_stats_df

In [ ]:
# get the average across all
average_number_of_tokens = np.mean(number_of_tokens_total)
sd_number_of_tokens = np.std(number_of_tokens_total)

average_number_of_tokens, sd_number_of_tokens

In [ ]:
# average across all models
average_number_of_tokens = model_token_stats_df["average_number_of_tokens"].mean()
sd_number_of_tokens = model_token_stats_df["sd_number_of_tokens"].mean()

average_number_of_tokens, sd_number_of_tokens

### LLM Evaluation Results

In [136]:
# the following files were manually created
claude_evaluator_results = pd.read_json("./pls_outputs/_interpretation_eval_results/claude_3.5-sonnet/claude_3.5-sonnet_interpretation_overall_metrics.json", orient="index")
gpt4o_mini_evaluator_results = pd.read_json("./pls_outputs/_interpretation_eval_results/gpt4o-mini/gpt4o-mini_interpretation_overall_metrics.json", orient="index")

In [ ]:
pls_gpt4o_mini_model_stats_df = calculate_model_stats(gpt4o_mini_evaluator_results, method_name = "GPT4o Mini")

pls_gpt4o_mini_model_stats_df

In [ ]:
pls_claude_model_stats_df = calculate_model_stats(claude_evaluator_results, method_name = "Claude 3.5 Sonnet")

pls_claude_model_stats_df

In [ ]:
# combine two dataframes
all_pls_model_stats_df = pd.concat([pls_gpt4o_mini_model_stats_df, pls_claude_model_stats_df], ignore_index=True)

all_pls_model_stats_df

In [ ]:
# create altair grouped barchart
# grouped by metric and evaluator

# Create a mapping for custom facet titles
facet_title_mapping = {
    'benefit_answer': 'Benefit',
    'rigor_answer': 'Rigor',
    'importance_answer': 'Importance',
    'full_text_answer': 'Full-Text',
    'another_trial_answer': 'Another Trial'
}

# Define the desired order for the facets
facet_order = ['Benefit', 'Rigor', 'Importance', 'Full-Text', 'Another Trial']

color_mapping = {
    'Claude 3.5 Sonnet': '#0868ac',  
    'GPT4o Mini': '#43a2ca',  
}

method_order = ['Claude 3.5 Sonnet', 'GPT4o Mini']

# Apply the mapping as a calculated field
chart_data = all_pls_model_stats_df.copy()
chart_data['metric'] = chart_data['metric'].map(facet_title_mapping)

# Configure global font sizes
chart_config = {
    "axis": {"labelFontSize": 20, "titleFontSize": 22},  # Axis labels and titles
    "header": {"labelFontSize": 20, "titleFontSize": 22},  # Facet headers
    "legend": {"labelFontSize": 18, "titleFontSize": 20},  # Legend labels and titles
    "text": {"fontSize": 20},  # Text mark size
}

# Bar chart
bars = alt.Chart(chart_data).mark_bar().encode(
    x=alt.X('evaluator:N', title=None, axis=alt.Axis(labelAngle=-45), sort = method_order),
    y=alt.Y('mean_diff:Q', title='Mean Difference'),
    color=alt.Color('evaluator:N', title='Evaluator', legend=None, scale=alt.Scale(domain=list(color_mapping.keys()), range=list(color_mapping.values())))
).properties(
    width=120,  # Set the width to 300 pixels
    height=250  # Set the height to 300 pixels
)

# Error bars
error_bars = alt.Chart(chart_data).mark_errorbar().encode(
    alt.X("evaluator:N", sort = method_order),
    alt.Y("ci_lower:Q").title("Mean Difference"),
    alt.Y2("ci_upper:Q"),
    strokeWidth=alt.value(2),
    color=alt.value('gray')
)

# Add value labels
text = bars.mark_text(
    align='center',
    baseline='bottom',
    fontWeight='bold',
    dy=alt.expr(expr=alt.expr.if_(alt.datum.mean_diff >= 0, -1, 20))  # Adjust the position of the text    
).encode(
    text=alt.Text('mean_diff:Q', format='.2f'),
    color=alt.value('black')  # Set text color to black
)

# Combine layers and facet
chart = alt.layer(bars, text, error_bars, data=chart_data).facet(
    column=alt.Column('metric:N', title=None, sort=facet_order),
).configure(**chart_config)  # Apply

# save to html
chart.save("./plots/pls_evaluator_comparison_by_measures.html")

chart

